# Unsupervised neural networks and word2vec

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
import gensim
from gensim.models import word2vec
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [4]:
# Parse the data. This can take some time.
nlp = spacy.load('en')
austen_doc = nlp(austen_clean)

In [5]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['daughter', 'eld', 'give', 'thing', 'tempt']
We have 8323 sentences and 900000 tokens.


In [6]:
model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [7]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('goddard', 0.9170064330101013), ('benwick', 0.9117452502250671), ('harville', 0.9114782214164734), ('musgrove', 0.8962116241455078), ('clay', 0.8950631618499756), ('wentworth', 0.8827566504478455), ('charles', 0.8426447510719299), ('croft', 0.8411537408828735), ('weston', 0.8360087871551514), ('colonel', 0.8321766257286072)]
0.8661102
marriage


## Drill 0

Take a few minutes to modify the hyperparameters of this model and see how its answers change. Can you wrangle any improvements?

In [13]:
# Tinker with hyperparameters here.
par_scores = []
for worker in [4, 6, 8, 10]:
    for count in [4, 6, 8, 10]:
        for wind in [4, 5, 6, 7]:
            for sz in [500, 1000, 1500, 2000]:
                model = word2vec.Word2Vec(
                    sentences,
                    workers=worker,     # Number of threads to run in parallel (if your computer does parallel processing).
                    min_count=count,  # Minimum word count threshold.
                    window=wind,      # Number of words around target word to consider.
                    sg=0,          # Use CBOW because our corpus is small.
                    sample=1e-3 ,  # Penalize frequent words.
                    size=sz,      # Word vector length.
                    hs=1           # Use hierarchical softmax.
                )
                par_scores.append([worker, count, wind, sz, model.wv.similarity('mr', 'mrs')])

                
# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
scores = [item[4] for item in par_scores]
score_ind = scores.index(max(scores))

print('Parameters for Model with best Similarity Score:')
print ('workers: {}\nmin_count: {}\nwindow: {}\nsize: {}'.format(par_scores[score_ind][0], par_scores[score_ind][1], par_scores[score_ind][2], par_scores[score_ind][3]))
print('Similarity Score: ', par_scores[score_ind][4])


Parameters for Model with best Similarity Score:
workers: 10
min_count: 10
window: 7
size: 2000
Similarity Score:  0.973688


## Drill 1: Word2Vec on 100B+ words

As we mentioned, word2vec really works best on a big corpus, but it can take half a day to clean such a corpus and run word2vec on it.  Fortunately, there are word2vec models available that have already been trained on _really_ big corpora. They are big files, but you can download a [pretrained model of your choice here](https://github.com/3Top/word2vec-api). At minimum, the ones built with word2vec (check the "Architecture" column) should load smoothly using an appropriately modified version of the code below, and you can play to your heart's content.

Because the models are so large, however, you may run into memory problems or crash the kernel. If you can't get a pretrained model to run locally, check out this [interactive web app of the Google News model](https://rare-technologies.com/word2vec-tutorial/#bonus_app) instead.

However you access it, play around with a pretrained model. Is there anything interesting you're able to pull out about analogies, similar words, or words that don't match? Write up a quick note about your tinkering and discuss it with your mentor during your next session.

In [ ]:
# Load Google's pre-trained Word2Vec model.
model = gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

After waiting several hours for the pretrained model to load in this Jupyter notebook, I still had no progress, so I elected instead to use the interactive web app of the Google news model, as provided above. Some interesting comparisons to note were:
 - banana orange Apple strawberry gives a result of Apple, because the model can differentiate apple the fruit from Apple the company
 - man is to woman as dude is to chick. This chose a female term that also has a similar style/ connotation as the term, dude, inputted.
 - USA matches Lifts_Squaw_Valley, but usa matches canada and U.S.A. matches Trademark_Lawsuit
 - Donald George Bill Joe returns George as the item that doesn't fit, but donald george bill joe returns bill, likely because the lowercase term can be a noun, instead of just a name